In [2]:
# ========== Step One -- Don't Modify Anything in This Cell
# Import Libraries and Modules
from Bio import SeqIO
from Bio.Seq import Seq
import subprocess
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [9]:
# ========== Step Two -- Don't Modify Anything in This Cell
# Define Functions
def fasta_to_string(fasta_path):
    return str(next(SeqIO.parse(fasta_path, "fasta")).seq)

def trim_reads_by_primer(fastq_filepath, primer):
    total_reads = []
    
    trimmed_reads = []

    primer = primer.upper()

    primer_len = len(primer)
    for rec in SeqIO.parse(fastq_filepath, "fastq"):
        seq = str(rec.seq)
        total_reads.append(seq)
        primer_idx = seq.find(primer)
        if primer_idx != -1:
            trimmed_seq = seq[primer_idx + primer_len:]
            trimmed_qual = rec.letter_annotations["phred_quality"][primer_idx + primer_len:]
            trimmed_reads.append((rec.id, trimmed_seq, trimmed_qual))
    
    if len(trimmed_reads) > 0:
        print(f"Total reads before trimming: {len(total_reads)}")
        return trimmed_reads

    else:
        print("Could not trim reads -- double check your primer sequence and offset.")

def write_trimmed_fastq(trimmed_reads, output_path):
    with open(output_path, "w") as f:
        for read_id, seq, qual in trimmed_reads:
            qual_str = ''.join([chr(q + 33) for q in qual])
            f.write(f"@{read_id}\n{seq}\n+\n{qual_str}\n")

def extract_insertion_sites(bam_file, strand="f"):
    result = subprocess.run(["samtools", "view", bam_file], capture_output=True, text=True, check=True)
    positions = []
    for line in result.stdout.strip().split("\n"):
        if not line or line.startswith('@'):
            continue
        fields = line.split("\t")
        pos = int(fields[3])
        read_len = len(fields[9])
        if strand == "f":
            positions.append(pos)
        else:
            positions.append(pos + read_len - 1)
    return positions

def sgRNA_finder(sgRNA, genome, PAM_len, natural_system):
    sgRNA = sgRNA.upper()
    genome = genome.upper()
    
    if natural_system:
        if genome.find(sgRNA) != -1:
            return genome.find(sgRNA), genome.find(sgRNA), genome.find(sgRNA) - PAM_len, False
        elif genome.find(str(Seq(sgRNA).reverse_complement())) != -1:
            return genome.find(str(Seq(sgRNA).reverse_complement())), genome.find(str(Seq(sgRNA).reverse_complement()))  + len(sgRNA), genome.find(str(Seq(sgRNA).reverse_complement())) + len(sgRNA) + PAM_len, True
        else:
            print("sgRNA Sequence Not Found in Genome")
    
    else:
        if genome.find(sgRNA) != -1:
            return genome.find(sgRNA), genome.find(sgRNA) + len(sgRNA), genome.find(sgRNA) + len(sgRNA) + PAM_len, False
        elif genome.find(str(Seq(sgRNA).reverse_complement())) != -1:
            return genome.find(str(Seq(sgRNA).reverse_complement())), genome.find(str(Seq(sgRNA).reverse_complement())), genome.find(str(Seq(sgRNA).reverse_complement())) - PAM_len, True
        else:
            print("sgRNA Sequence Not Found in Genome")

def integration_finder(insertion_idx_list, ccdb_gene, ccdb_start_site, PAM_start_site, PAM_end_site, natural_system, is_rc):
    dist_list = []

    if natural_system:
        if is_rc:
            def dist_calc(idx): return (PAM_start_site - idx)

        else:
            def dist_calc(idx): return (idx - PAM_start_site)
    
    else:
        if is_rc:
            def dist_calc(idx): return (PAM_end_site - idx)

        else:
            def dist_calc(idx): return (idx - PAM_end_site)

    for idx in insertion_idx_list:
        if ccdb_start_site + len(ccdb_gene) >= idx >= ccdb_start_site:
            modified_idx = dist_calc(idx)
            dist_list.append(modified_idx)

    return dist_list

def plot_insertion_histogram(
    forward_positions, reverse_positions,
    region_start=0, region_end=None,
    bins=100, title="Insertion Sites",
    highlight_regions=None,
    filename=None
):

    if region_end is None:
        region_end = max(forward_positions + reverse_positions)

    plt.figure(figsize=(10, 4))
    ax = plt.gca()

    # Plot histograms
    ax.hist(forward_positions, bins=bins, range=(region_start, region_end),
            alpha=0.5, label='Top Strand', color='black')
    ax.hist(reverse_positions, bins=bins, range=(region_start, region_end),
            alpha=0.5, label='Bottom Strand', color='grey')

    # Plot horizontal red bars
    if highlight_regions:
        ymin, _ = ax.get_ylim()
        y = ymin - 0.01 * (ax.get_ylim()[1] - ymin)
        for region in highlight_regions:
            if len(region) == 3:
                x_start, x_end, color = region
            else:
                x_start, x_end = region
                color = 'red'
            ax.hlines(y=2, xmin=x_start, xmax=x_end, colors=color, linewidth=5)

    ax.set_xlabel("Genomic Position")
    ax.set_ylabel("Insertion Count")
    ax.set_title(title)
    ax.set_xlim(region_start, region_end)
    ax.legend()
    plt.tight_layout()

    if filename:
        plt.savefig(filename, dpi=300)
        plt.close()
    else:
        plt.show()

def plot_integration_profile(data, title="Integration Distance From PAM", xlabel="Distance From PAM (bp)", ylabel="Count", 
                   xlabel_fontsize=12, ylabel_fontsize=12, title_fontsize=14, 
                   hist_color='grey', edge_color='black', linewidth=5, dpi=300, xlim=(20,100), filename=None):
    
    plt.clf()
    plt.hist(data, edgecolor=edge_color, color=hist_color, linewidth=linewidth)
    plt.title(title, fontsize=title_fontsize)
    plt.xlabel(xlabel, fontsize=xlabel_fontsize)
    plt.ylabel(ylabel, fontsize=ylabel_fontsize)
    
    if xlim:
        plt.xlim(xlim)
    
    plt.gcf().set_dpi(dpi)
    
    plt.tick_params(axis='both', labelsize=10)

    if filename:
        plt.savefig(filename, dpi=dpi)
        plt.close()
    else:
        plt.show()

In [10]:
# ========== Step Three -- Modify Parameters for Your Use Case

# Parameters to Change

# Absolute filepath to your merged paired end reads in a .fastq file format
fastq_filepath = "/Volumes/groups/kellogrp/home/common/asmiley/NGS/PrashantTagmentation/PD2_merged.fastq"

# Primer sequence to the end of the LE or RE
primer = "CTGAAAAACAACCACCACGACATTAATTTGCGAATAACGACACTAAATTGCGAAAAGCGACATTTAATTTGCGAATGTACA"
# AcCAST LE CTGAAAAACAACCACCACGACATTAATTTGCGAATAACGACACTAAATTGCGAAAAGCGACATTTAATTTGCGAATGTACA
# PmcCAST LE CCCTTTATTGGAAGCATAAGCTTGCCGTTGCGGCAAAGTTATGGGTAAAGTCACA

# Absolute filepath to CJP003 genome .fasta file
referencegenome_filepath = "/Volumes/groups/kellogrp/home/common/06_ProjectStorage/NovelCAST/ProfilingData/Tagmentation/cJP003_assembly.fa"

# The first 10 bases in your donor plasmid immediately following the LE or RE -- used to throw away non-enzymatic integrations/recombinations
first_ten_in_donor = "TATATAATGG"
# AcCAST TATATAATGG
# PmcCAST ATCCCAATGG

# Sequence of the targeting portion of your sgRNA
sgRNA_sequence = "TTTTTGGGCTAGCGATTGAAAACG"

# PmcCAST De novo sgRNAs
# Spacer 1 TGAAAGCTGGCGCATGATGACCACCGATA
# Spacer 2 TGAACTTTACCCGGTGGTGCATATCGGGGA
# Spacer 3 ACTTTATCTGACAGCAGACGTGCACTGGCC
# Spacer 4 AGGTTAATGGCGTTTTTGATGTCATTTTCG

# AcCAST sgRNAs
# Spacer 1 TCTCCATACCCGTTTTTTTGGGCT
# Spacer 2 TTTTTGGGCTAGCGATTGAAAACG
# Spacer 3 TACACCTATAAAAGAGAGAGCCGT

# Length of the CAST's PAM Sequence
PAM_len = 3

# Is this a naturally occuring CAST system?
natural_system = True # Set to false if Novel CAST -- integration site relative to the PAM is different than in natural systems

# Filepath for Output files from bbmap -- change to the absolute filepath where you want them exported to, end a dir with "/"
path_to_output_dir = "/Users/asmiley/Downloads/"

# Adds this to the beginning of each output file for simplicity
sample_abbreviation = "PD2"

# Names of files of interest -- change to your liking
trimmed_fastq = f"{sample_abbreviation}_trimmed_reads.fastq"
ForwardInsertions = f"{sample_abbreviation}_forward_insertions.txt"
ReverseInsertions = f"{sample_abbreviation}_reverse_insertions.txt"
genome_wide_integrations = f"{sample_abbreviation}_genome_insertion_histogram.png"
ccdb_integrations = f"{sample_abbreviation}_CCDB_insertion_histogram.png"
integration_distances = f"{sample_abbreviation}_Integration_profile.png"
integration_report = f"{sample_abbreviation}_integration_report.txt"

# Useful variables -- only change if necessary
genome_sequence = fasta_to_string(referencegenome_filepath)
ccdb_gene = "ATGCAGTTTAAGGTTTACACCTATAAAAGAGAGAGCCGTTATCGTCTGTTTGTGGATGTACAGAGTGATATTATTGACACGCCCGGGCGACGGATGGTGATCCCCCTGGCCAGTGCACGTCTGCTGTCAGATAAAGTCTCCCGTGAACTTTACCCGGTGGTGCATATCGGGGATGAAAGCTGGCGCATGATGACCACCGATATGGCCAGTGTGCCGGTCTCCGTTATCGGGGAAGAAGTGGCTGATCTCAGCCACCGCGAAAATGACATCAAAAACGCCATTAACCTGATGTTTTGGGGAATATAA"
ccdb_start_site = genome_sequence.find(ccdb_gene)
sgRNA_start_site, PAM_start_site, PAM_end_site, is_rc = sgRNA_finder(sgRNA_sequence, genome_sequence, PAM_len, natural_system)

highlight = [
    (sgRNA_start_site, sgRNA_start_site + len(sgRNA_sequence), 'red'), (PAM_start_site, PAM_end_site, 'black')
]


In [ ]:
# ========== Step Four -- Don't Modify Anything in This Cell

# 1. Trim reads by primer
trimmed_reads = trim_reads_by_primer(fastq_filepath, primer)

# 2. Remove donor reads
donor_removed_reads = [read for read in trimmed_reads if not read[1].startswith(first_ten_in_donor)]
print(f"Total reads after primer trimming: {len(trimmed_reads)}")
print(f"Total reads after donor removal: {len(donor_removed_reads)}")

# 3. Write output FASTQ with original qualities preserved
trimmed_fastq = Path(f"{path_to_output_dir}{trimmed_fastq}")
write_trimmed_fastq(donor_removed_reads, trimmed_fastq)

# 4. BBMap indexing and alignment
subprocess.run(["bbmap.sh", f"ref={referencegenome_filepath}"], check=True)
mapped_bam = trimmed_fastq.with_suffix(".bam")
subprocess.run([
    "bbmap.sh", f"in={trimmed_fastq}", f"outm={mapped_bam}",
    "minid=0.9", "ambig=random"
], check=True)

# 5. Separate forward/reverse BAM
forward_bam = mapped_bam.with_name(mapped_bam.stem + "_f.bam")
reverse_bam = mapped_bam.with_name(mapped_bam.stem + "_r.bam")
subprocess.run(f"samtools view -F 0x10 -h {mapped_bam} | samtools view -bS - > {forward_bam}", shell=True, check=True)
subprocess.run(f"samtools view -f 0x10 -h {mapped_bam} | samtools view -bS - > {reverse_bam}", shell=True, check=True)

# 6. Proper insertion site extraction
forward_insertions = extract_insertion_sites(forward_bam, "f")
reverse_insertions = extract_insertion_sites(reverse_bam, "r")

# Save positions
Path(f"{path_to_output_dir}{ForwardInsertions}").write_text("\n".join(map(str, forward_insertions)))
Path(f"{path_to_output_dir}{ReverseInsertions}").write_text("\n".join(map(str, reverse_insertions)))

# Plot Integration profiles in genome and in CCDB gene
plot_insertion_histogram(forward_insertions, reverse_insertions, region_start=0, region_end=len(genome_sequence), title="Genome", highlight_regions=highlight, filename=f"{path_to_output_dir}{genome_wide_integrations}")
plot_insertion_histogram(forward_insertions, reverse_insertions, region_start=ccdb_start_site-50, region_end=ccdb_start_site+len(ccdb_gene), title="CCDB Gene", highlight_regions=highlight, filename=f"{path_to_output_dir}{ccdb_integrations}")

# Plot Integration Profiles
modified_forward_insertions = integration_finder(forward_insertions, ccdb_gene, ccdb_start_site, PAM_start_site, PAM_end_site, natural_system, is_rc)

modified_reverse_insertions = integration_finder(reverse_insertions, ccdb_gene, ccdb_start_site, PAM_start_site, PAM_end_site, natural_system, is_rc)

combined_integrations = modified_forward_insertions + modified_reverse_insertions

plot_integration_profile(combined_integrations, filename=f"{path_to_output_dir}{integration_distances}")

# --- Calculate integration stats ---
fwd_total = len(forward_insertions)
rev_total = len(reverse_insertions)
fwd_on_target = len(modified_forward_insertions)
rev_on_target = len(modified_reverse_insertions)
total = fwd_total + rev_total
on_target_total = fwd_on_target + rev_on_target

fwd_pct = 100 * fwd_on_target / fwd_total if fwd_total else 0
rev_pct = 100 * rev_on_target / rev_total if rev_total else 0
total_pct = 100 * on_target_total / total if total else 0

all_lengths = modified_forward_insertions + modified_reverse_insertions
mean_len = np.mean(all_lengths) if all_lengths else 0
median_len = np.median(all_lengths) if all_lengths else 0
std_len = np.std(all_lengths) if all_lengths else 0

# --- Format and write report ---
report_text = f"""Integration Analysis Report
==================================
Total insertions:
  Forward strand: {fwd_total}
  Reverse strand: {rev_total}
  Combined total: {total}

On-target insertions (within CCDB region):
  Forward strand: {fwd_on_target} ({fwd_pct:.2f}%)
  Reverse strand: {rev_on_target} ({rev_pct:.2f}%)
  Combined on-target: {on_target_total} ({total_pct:.2f}%)

Integration length statistics (bp):
  Mean length:   {mean_len:.2f}
  Median length: {median_len:.2f}
  Std deviation: {std_len:.2f}
"""

# Create output directory if needed
output_report_path = Path(f"{path_to_output_dir}{integration_report}")
output_report_path.parent.mkdir(parents=True, exist_ok=True)
output_report_path.write_text(report_text)

print(f"Integration report written to '{output_report_path.resolve()}'")